In [51]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta

In [52]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\output data'
namedataASCD_M = r'dataASCD_M.xlsx'
rutedataASCD_M = os.path.join(folder,namedataASCD_M)

In [53]:
dataASCD_M = pd.read_excel(rutedataASCD_M,header=0)

In [54]:
filterMT = dataASCD_M.loc[:,'Maintenance type'] != 'COND'
dataASCD_1TM = dataASCD_M.loc[filterMT,:].reset_index(drop=True)

In [55]:
# Adding the pivot column to project
dataASCD_1TM.loc[:,'pivot'] = dataASCD_1TM.loc[:,'Asset tag'] + '-' + dataASCD_1TM.loc[:,'Strategy'] 
#dataASCD_1TP.loc[:,'pivot'] = dataASCD_1TP.loc[:,'Asset tag'] + '-' + dataASCD_1TP.loc[:,'Strategy']

In [56]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo','Strategy', 'Date task done']

In [62]:
projectionM = dataASCD_1TM.loc[:,['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']].groupby(['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']).size().reset_index()
projectionM.drop(columns = {0},inplace =True)
projectionM.loc[:,'Date plan'] = projectionM.loc[:,'Last date task done']
[row,column] = projectionM.shape

In [115]:
A = pd.DataFrame(projectionM.loc[:,'Strategy'].unique(),columns=['Strategy'])
A

,Strategy
0,1000H
1,165H
2,4000H
3,660H
4,8000H
5,1A
6,1M
7,6M
8,2000H
9,1400H


In [108]:
B = pd.DataFrame(pd.date_range(start= date(2021,4,1),end=date(2021,4,9),freq='1d'),columns = ['date'])

In [110]:
B

,date
0,2021-04-01
1,2021-04-02
2,2021-04-03
3,2021-04-04
4,2021-04-05
5,2021-04-06
6,2021-04-07
7,2021-04-08
8,2021-04-09


In [112]:
C = pd.merge(A,B,how='left',left_on='Strategy',right_on='date')

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

In [92]:
%%time
planDatesM = pd.DataFrame(columns = projectionM.columns)
projectionflag = projectionM.copy()
for m in range(row):
    while projectionflag.loc[m,'Date plan'] < date(2023,1,1):
        hour2days = round(projectionflag.loc[m,'Frequency']/projectionflag.loc[m,'Hour day'],0)
        if projectionflag.loc[m,'Unidad Periodo'] == 'H':
            delta=timedelta(days = hour2days)
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'S':
            delta = timedelta(weeks = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'M':
            delta = timedelta(weeks = 4*projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'A':
            delta = timedelta(weeks = 52*projectionflag.loc[m,'Frequency'])
        projectionflag.loc[m,'Date plan'] = projectionflag.loc[m,'Date plan'] + delta
        if projectionflag.loc[m,'Date plan'] < date(2023,1,1):
            planDatesM = planDatesM.append(projectionflag.loc[m,:],ignore_index=True)
        else:
            pass
planDatesM.loc[:,'pivot'] = planDatesM.loc[:,'Asset tag'] + '-' + planDatesM.loc[:,'Strategy']

Wall time: 56.3 s


In [95]:
planDatesM.shape

(13221, 8)

In [93]:
%%time
dataASCD_1TMPlan = pd.merge(dataASCD_1TM,planDatesM.loc[:,['pivot','Date plan']],how='left',on='pivot')
dataASCD_1TMPlan.to_excel('dataASCD_1TMPlan.xlsx', encoding='utf-8-sig',index=False)

Wall time: 40.9 s


In [94]:
dataASCD_1TMPlan.shape

(63305, 30)